see https://help.marine.copernicus.eu/en/articles/6725319-how-to-download-data-for-a-recurring-period-over-several-years-via-the-motu-client-in-python 
for howto make this cli command into a program

In [56]:
# Base command
# python -m motuclient --motu https://my.cmems-du.eu/motu-web/Motu
# --service-id WIND_GLO_PHY_L4_MY_012_006-TDS
# --product-id cmems_obs-wind_glo_phy_my_l4_0.125deg_PT1H
# --longitude-min -12.455343303571427 --longitude-max 16.473195982142855
# --latitude-min 45.241023613016935 --latitude-max 62.91957539873122
# --date-min "2021-01-01 00:00:00" --date-max "2021-01-31 12:00:00"
# --variable eastward_wind --variable northward_wind
# --out-dir ./ --out-name test.nc --user fborg --pwd Zx7iwpWtqKqPTCG

In [57]:
import os
from datetime import datetime, timedelta
import getpass
import xarray as xr

In [59]:
# Copernicus Marine Credentials 
USERNAME = input('Enter your username: ')
PASSWORD = getpass.getpass("Enter your password: ")

# Work directory
out_dir = '/data/exjobb/sarssw/model/WIND_GLO_PHY'

# product and dataset IDs
# For https://data.marine.copernicus.eu/product/WIND_GLO_PHY_L4_MY_012_006/description
serviceID = "WIND_GLO_PHY_L4_MY_012_006"
productID = "cmems_obs-wind_glo_phy_my_l4_0.125deg_PT1H"

#coordinates
lon = (-12 , 16.5)
lat = (45 , 63)

# variable 
vars = ["eastward_wind", "northward_wind"]

# Boundary dates
sec = timedelta(seconds=1)
start_dates = [datetime(2021, i, 1, 0) for i in range(1, 13)]
end_dates = [d - sec for d in start_dates[1:]] + [datetime(2021, 12, 31, 23)]
date_intervals = list(zip(start_dates, end_dates))

In [61]:
#Conditionally creates the folder for the result
os.makedirs(out_dir, exist_ok=True)

# Download loop 
for i, (start_date, end_date) in enumerate(date_intervals):

    # Output filename
    out_name = f"{i}.nc"
    
    # Motuclient command line
    query = f'python -m motuclient --motu https://my.cmems-du.eu/motu-web/Motu \
    --service-id {serviceID}-TDS --product-id {productID} \
    --longitude-min {lon[0]} --longitude-max {lon[1]} --latitude-min {lat[0]} --latitude-max {lat[1]}\
    --date-min "{start_date}" --date-max "{end_date}" \
    {" ".join([f"--variable {v}" for v in vars])} \
    --out-dir {out_dir} --out-name {out_name} --user {USERNAME} --pwd {PASSWORD}'        

    print(f"============== Running request on {start_date} ==============")
    #print(query)
    
    # Run the command
    os.system(query)

        
print(f"============== Download completed! All files are in your directory {out_dir} ==============")



============== Running request on 2021-01-01 00:00:00 ==============
2023-03-03 11:27:03.803 [ INFO] Asynchronous mode set
2023-03-03 11:27:03.804 [ INFO] Authenticating user fborg for service https://my.cmems-du.eu/motu-web/Motu
2023-03-03 11:27:04.623 [ INFO] Requesting file to download (this can take a while)...
2023-03-03 11:27:10.340 [ INFO] Authenticating user fborg for service https://my.cmems-du.eu/motu-web/Motu
2023-03-03 11:27:16.556 [ INFO] Product is not yet available (request in progress)
2023-03-03 11:27:26.566 [ INFO] Authenticating user fborg for service https://my.cmems-du.eu/motu-web/Motu
2023-03-03 11:27:33.699 [ INFO] Product is not yet available (request in progress)
2023-03-03 11:27:43.709 [ INFO] Authenticating user fborg for service https://my.cmems-du.eu/motu-web/Motu
2023-03-03 11:27:49.958 [ INFO] Product is not yet available (request in progress)
2023-03-03 11:27:59.961 [ INFO] Authenticating user fborg for service https://my.cmems-du.eu/motu-web/Motu
2023-0

In [65]:
#Test load one dataset
cop_wind_ds = xr.open_dataset(os.path.join(out_dir, file_list[0]))
cop_wind_ds

<xarray.Dataset>
Dimensions:         (time: 744, lat: 145, lon: 229)
Coordinates:
  * lon             (lon) float32 -12.06 -11.94 -11.81 ... 16.19 16.31 16.44
  * time            (time) datetime64[ns] 2021-10-01 ... 2021-10-31T23:00:00
  * lat             (lat) float32 44.94 45.06 45.19 45.31 ... 62.69 62.81 62.94
Data variables:
    northward_wind  (time, lat, lon) float32 ...
    eastward_wind   (time, lat, lon) float32 ...
Attributes: (12/29)
    title:                                           Global Ocean - Wind and ...
    summary:                                        Global ocean 10-m stress-...
    keywords:                                       ocean winds, wind speed, ...
    Conventions:                                    CF-1.6, ACDD-1.3
    project:                                        Copernicus Marine Service...
    institution:                                    Royal Netherlands Meteoro...
    ...                                             ...
    history:                                        N/A
    date_created:                                   2023-01-17T02:56:07
    date_modified:                                  2023-01-17T02:56:07
    _CoordSysBuilder:                               ucar.nc2.dataset.conv.CF1...
    comment:                                        
    source:

In [76]:
#Add record variable (time) to all files
int_file_list_sorted = sorted([int(f[:-3]) for f in file_list if f[-3:] == '.nc'])
for file_int in int_file_list_sorted:
    cmd = f"ncks --mk_rec_dmn time {os.path.join(out_dir, str(file_int))}.nc -o {os.path.join(out_dir, str(file_int))}_rec.nc"
    #print(cmd)
    # Run the command
    os.system(cmd)

In [81]:
#Join the datasets
cmd = f"ncrcat {' '.join([os.path.join(out_dir, str(file_int)) + '_rec.nc' for file_int in int_file_list_sorted])} {os.path.join(out_dir,'all.nc')}"
#print(cmd)
# Run the command
os.system(cmd)

ncrcat: INFO/WARNING Multi-file concatenator encountered packing attribute scale_factor for variable eastward_wind. NCO copies the packing attributes from the first file to the output file. The packing attributes from the remaining files must match exactly those in the first file or data from subsequent files will not unpack correctly. Be sure all input files share the same packing attributes. If in doubt, unpack (with ncpdq -U) the input files, then concatenate them, then pack the result (with ncpdq). This message is printed only once per invocation.


0